In [216]:
from glob import glob
import pandas
from datetime import datetime
import numpy

將原始資料解析成正確形式
"-"的處理,避免每一欄都是文字
<h3>任務->產生SQL語句</h3>

In [217]:
data_path='/Users/justin/Desktop/資料庫管理/data/W06_data'
file_list=glob('%s/NQF-*.txt'% data_path)
#會進行代換*
file_list

['/Users/justin/Desktop/資料庫管理/data/W06_data/NQF-2017.txt',
 '/Users/justin/Desktop/資料庫管理/data/W06_data/NQF-2019.txt',
 '/Users/justin/Desktop/資料庫管理/data/W06_data/NQF-2018.txt',
 '/Users/justin/Desktop/資料庫管理/data/W06_data/NQF-2020.txt']

In [218]:
datatable='StockInfo'

In [219]:
data=pandas.read_csv(file_list[0],delimiter= '\t')

In [220]:
data

,Date,Open,High,Low,Close,AdjClose,Volume
0,"Dec 29, 2017","6,444.50","6,467.75","6,383.25","6,408.75","6,408.75","210,114"
1,"Dec 28, 2017","6,452.25","6,477.50","6,442.25","6,446.25","6,446.25","138,254"
2,"Dec 27, 2017","6,461.25","6,475.50","6,442.00","6,454.00","6,454.00","158,481"
3,"Dec 26, 2017","6,482.50","6,492.00","6,432.25","6,460.00","6,460.00","135,130"
4,"Dec 22, 2017","6,489.75","6,501.75","6,467.50","6,483.75","6,483.75","130,810"
...,...,...,...,...,...,...,...
246,"Jan 09, 2017","5,004.50","5,029.50","5,002.50","5,022.00","5,022.00","148,112"
247,"Jan 06, 2017","4,962.50","5,017.50","4,952.50","5,004.00","5,004.00","183,315"
248,"Jan 05, 2017","4,934.75","4,964.25","4,920.50","4,962.00","4,962.00","169,565"
249,"Jan 04, 2017","4,906.50","4,942.50","4,903.00","4,933.50","4,933.50","171,952"


<h3>工具函數-依據輸入的不同做執行</h3>

欄位間格為tab(\t),
將[0]解析為單獨的日期列(處理空格問題),
推斷日期時間字符串的格式,
header=0表示有檔頭,並將欄位名稱做相對應的代換

新增衍生欄位stockID,
由小到大排列,並將產生的index拆掉(drop),內部結構改變(inpalce)

利用loc尋找沒有"-"的 資料(將-刪掉)

定義數值欄,
apply各個欄位都執行一件事:轉成字串,將","replace掉,再轉為數值

<h3>函數導向設計</h3>.
lambda x:傳回變數,內容可代換(自行定義)->第二次期中

In [221]:
def import_data(stockID, file_name):
    _dataset = pandas.read_csv(
        file_name, delimiter='\t', parse_dates=True, infer_datetime_format=True,
        header=0, names=['StockDate', 'StockOpen', 'StockHigh', 
                         'StockLow', 'StockClose', 'StockAdjClose', 'StockVol'],
    )
    
    _dataset['StockID'] = stockID
    _dataset = _dataset.sort_values(by=['StockDate'])
    
    _dataset.reset_index(drop=True, inplace=True)
    
    _dataset = _dataset.loc[_dataset['StockOpen'] != '-']
    
    _number_fields = ['StockOpen', 'StockHigh', 'StockLow', 'StockClose', 
                      'StockAdjClose', 'StockVol']
    
    _dataset[_number_fields] = _dataset[_number_fields].apply(
        lambda x: pandas.to_numeric(x.astype(str).str.replace(',', ''), errors='coerce')
    )
    
    _dataset['StockVol'] = _dataset['StockVol'] / 1000.0
            
    return _dataset[['StockID', 'StockDate', 
                    'StockOpen', 'StockHigh', 'StockLow', 'StockClose', 
                    'StockAdjClose', 'StockVol']]

<h3>載入檔案</h3>

In [222]:
_dataset=import_data('NQF',file_list[0])
#呼叫工具程式傳入"(NQF,檔名)"

In [223]:
_dataset.tail()

,StockID,StockDate,StockOpen,StockHigh,StockLow,StockClose,StockAdjClose,StockVol
246,NQF,"Sep 25, 2017",5930.75,5954.25,5842.00,5876.5,5876.5,447.828
247,NQF,"Sep 26, 2017",5878.75,5912.50,5862.75,5892.5,5892.5,343.472
248,NQF,"Sep 27, 2017",5889.75,5963.00,5886.50,5941.0,5941.0,349.282
249,NQF,"Sep 28, 2017",5936.00,5948.25,5911.25,5938.5,5938.5,268.097
250,NQF,"Sep 29, 2017",5939.25,5985.50,5933.00,5982.5,5982.5,275.661


<h3>建立表格SQL命令搞(樣板)</h3>
用pandas函數將datasets傳入,並給一個table,用換行鍵(\n)拆分,以向量形式輸出

In [224]:
_create_table_sql=pandas.io.sql.get_schema(_dataset,datatable).split('\n')
_create_table_sql

['CREATE TABLE "StockInfo" (',
 '"StockID" TEXT,',
 '  "StockDate" TEXT,',
 '  "StockOpen" REAL,',
 '  "StockHigh" REAL,',
 '  "StockLow" REAL,',
 '  "StockClose" REAL,',
 '  "StockAdjClose" REAL,',
 '  "StockVol" REAL',
 ')']

In [225]:
def StringtoDatetime(x):
    date=datetime.strptime(x,"%b %d, %Y")
    date=datetime.strftime(date,'%Y-%m-%d')
    return date

<h3>新增筆錄至表格內SQL命令搞</h3>

In [226]:
_sql = 'INSERT INTO %s ' % datatable
_sql = _sql + '(%s, %s, %s, %s, %s, %s, %s, %s) ' % tuple(_dataset.columns)
#tuple格式化分配

_insert_table_sql_list = []

#用迴圈將每筆資料放入陣列,存入DB
for i in range(_dataset.shape[0]):
    #shape0是維度,表示資料有幾筆
    #shape1表示欄位
    _row = _dataset.iloc[i]
    #第幾筆資料
    _insert_table_sql = _sql + "VALUES('%s','%s',%s,%s,%s,%s,%s,%s)" % (
        _row['StockID'], StringtoDatetime(_row['StockDate']), #字串轉日期 
        _row['StockOpen'], _row['StockHigh'], _row['StockLow'], _row['StockClose'],
        _row['StockAdjClose'], _row['StockVol']
    )    
    
    _insert_table_sql_list.append(_insert_table_sql)

_insert_table_sql_list[0]

"INSERT INTO StockInfo (StockID, StockDate, StockOpen, StockHigh, StockLow, StockClose, StockAdjClose, StockVol) VALUES('NQF','2017-04-03',5436.75,5455.0,5403.25,5434.5,5434.5,264.74)"

In [227]:
_insert_table_sql_list[2]

"INSERT INTO StockInfo (StockID, StockDate, StockOpen, StockHigh, StockLow, StockClose, StockAdjClose, StockVol) VALUES('NQF','2017-04-05',5447.0,5481.25,5409.0,5416.25,5416.25,301.129)"

In [228]:
#單筆資料寫法
_insert_table_sql= _sql+"VALUES(%s, %s, %s, %s, %s, %s, %s, %s)"%(
    'NQF','2020-12-24',-100.0,-100.0,-100.0,-100.0,-100.0,-100.0
    )
_insert_table_sql

'INSERT INTO StockInfo (StockID, StockDate, StockOpen, StockHigh, StockLow, StockClose, StockAdjClose, StockVol) VALUES(NQF, 2020-12-24, -100.0, -100.0, -100.0, -100.0, -100.0, -100.0)'

<h3>異動筆錄至表格內SQL命令稿</h3>

In [229]:
_update_table_sql="UPDATE %s SET %s=%s WHERE StockID='%s' AND StockDate ='%s'" %(
    datatable,'StockVol',0.0,'NQF','2020-12-24')
_update_table_sql

"UPDATE StockInfo SET StockVol=0.0 WHERE StockID='NQF' AND StockDate ='2020-12-24'"

<h3>刪除筆錄至表格內SQL命令稿</h3>

In [230]:
_delete_table_sql="DELETE FROM %s WHERE StockID='%s'AND StockDate='%s'"%(
    datatable, 'NQF', '2020-12-24')
_delete_table_sql

"DELETE FROM StockInfo WHERE StockID='NQF'AND StockDate='2020-12-24'"